In [15]:
import pandas as pd

In [16]:
df_q = pd.read_json('v2_Questions_Train_mscoco/v2_OpenEnded_mscoco_train2014_questions.json')
df_a = pd.read_json('v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json')

In [17]:
df_q.head()

,image_id,question,question_id
0,458752,What is this photo taken looking through?,458752000
1,458752,What position is this man playing?,458752001
2,458752,What color is the players shirt?,458752002
3,458752,Is this man a professional baseball player?,458752003
4,262146,What color is the snow?,262146000


In [18]:
df_a.head()

,question_type,multiple_choice_answer,answers,image_id,answer_type,question_id
0,what is this,net,"[{'answer': 'net', 'answer_confidence': 'maybe...",458752,other,458752000
1,what,pitcher,"[{'answer': 'pitcher', 'answer_confidence': 'y...",458752,other,458752001
2,what color is the,orange,"[{'answer': 'orange', 'answer_confidence': 'ye...",458752,other,458752002
3,is this,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",458752,yes/no,458752003
4,what color is the,white,"[{'answer': 'white', 'answer_confidence': 'yes...",262146,other,262146000


In [19]:
# df_a = df_a.drop('question_id', axis=1)
df_QA =  pd.merge(df_q, df_a, on=['question_id','image_id'], how='inner')
df_QA.head(15)

,image_id,question,question_id,question_type,multiple_choice_answer,answers,answer_type
0,458752,What is this photo taken looking through?,458752000,what is this,net,"[{'answer': 'net', 'answer_confidence': 'maybe...",other
1,458752,What position is this man playing?,458752001,what,pitcher,"[{'answer': 'pitcher', 'answer_confidence': 'y...",other
2,458752,What color is the players shirt?,458752002,what color is the,orange,"[{'answer': 'orange', 'answer_confidence': 'ye...",other
3,458752,Is this man a professional baseball player?,458752003,is this,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",yes/no
4,262146,What color is the snow?,262146000,what color is the,white,"[{'answer': 'white', 'answer_confidence': 'yes...",other
5,262146,What is the person doing?,262146001,what is the person,skiing,"[{'answer': 'skiing', 'answer_confidence': 'ye...",other
6,262146,What color is the persons headwear?,262146002,what color is the,red,"[{'answer': 'red', 'answer_confidence': 'yes',...",other
7,524291,What is in the person's hand?,524291000,what is in the,frisbee,"[{'answer': 'frisbee', 'answer_confidence': 'y...",other
8,524291,Is the dog waiting?,524291001,is the,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",yes/no
9,524291,Is the dog looking at a tennis ball or frisbee?,524291002,is the,frisbee,"[{'answer': 'frisbee', 'answer_confidence': 'y...",other


In [20]:
df_QA['multiple_choice_answer'] = df_QA.apply(lambda col: "It's " + col['multiple_choice_answer'] if col['answer_type'] == 'other' else col['multiple_choice_answer'], axis = 1)
df_QA['multiple_choice_answer']

0                              It's net
1                          It's pitcher
2                           It's orange
3                                   yes
4                            It's white
                      ...              
443752                       It's black
443753                               no
443754                       It's black
443755    It's one is easier to type on
443756                              yes
Name: multiple_choice_answer, Length: 443757, dtype: object

In [21]:
df_QA['question'] = df_QA['question'].map(lambda x: '<user>' + x+'</user>')
df_QA['multiple_choice_answer'] = df_QA['multiple_choice_answer'].map(lambda x: '<assist>' + x+'</assist>')
df_QA['system_prompt'] = "<sys>You are a useful Visual Assitant</sys>"


In [22]:
df_QA['prompt'] = df_QA.apply(lambda row: f"{row['system_prompt']}{row['question']}{row['multiple_choice_answer']}", axis=1)

In [23]:
df_final = df_QA[['image_id','prompt']]
df_final.head()

,image_id,prompt
0,458752,<sys>You are a useful Visual Assitant</sys><us...
1,458752,<sys>You are a useful Visual Assitant</sys><us...
2,458752,<sys>You are a useful Visual Assitant</sys><us...
3,458752,<sys>You are a useful Visual Assitant</sys><us...
4,262146,<sys>You are a useful Visual Assitant</sys><us...


In [24]:
df_final.to_csv('output.csv', index=False)